In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
from sklearn.metrics import r2_score
import seaborn as sns

from tqdm import tqdm

In [2]:
import nibabel as nb
from nilearn import plotting, image
from nipype.interfaces import fsl

In [3]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/Users/sungbeenpark/github'

In [4]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [5]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [6]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [7]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [8]:
list_sn = su.get_list_sn()
list_roi = simage.get_list_roi()
list_run = su.get_list_run()

---

In [9]:
border, brdr = simage.get_border(join(dir_git,'surfAnalysisPy'))
hem = 'L'

---

In [10]:
glm = 1
dir_glm = join(su.get_dir_glm(glm))

---

In [34]:
for ii, sn in enumerate(tqdm(list_sn)):
    subj = 'S'+sn
    dir_work = join(dir_glm,subj)
    
    reginfo = deal_spm.load_reginfo(subj,dir_glm)
    
    mask = simage.load_mask(subj,glm)
    affine = mask.affine
    header = mask.header
    
    fnames = [join(dir_work,'beta_%04d.nii'%(idx+1)) for idx in reginfo.index]
    
    ResMS = nb.load(join(dir_work,'ResMS.nii'))
    for fname in fnames:
        beta = nb.load(fname)
        beta_new = beta.get_fdata()/(np.sqrt(ResMS.get_fdata())+1.e-14)
        
        img = nb.Nifti1Image(beta_new, affine, header)
        fname_new = fname.split('/')[-1].split('.')[0] + '.prewhiten.nii'
        nb.save(img, join(dir_work,fname_new))

100%|█████████████████████████████████████████████████████████████████████████| 12/12 [03:15<00:00, 16.29s/it]


---